In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def homogeneous_coords(xy):
    """Return copy of xy with additional dimension (==1).

    Parameters
    ----------
    xy : np.array of shape (n,) or (m, n)
        n dimensions, m values

    Returns
    -------
    np.array of shape (n+1,) or (m, n+1)
        Additional dimension with constant value == 1.
    """
    if xy.ndim == 1:
        newshape = (xy.shape[0]+1, )
        newxy = np.ndarray(shape=newshape)
        newxy[:-1] = xy
        newxy[-1] = 1
        return newxy

    newshape = xy.shape[0], xy.shape[1]+1
    newxy = np.ndarray(shape=newshape)
    newxy[:, :xy.shape[1]] = xy

    newxy[:, -1] = 1
    return newxy

def linear_transformation(a, b):
    """Least squares fit linear transformations of points a to b.

    Parameters
    ----------
    a, b : np.array of shape (n, 2)
        x- and y-coords

    Returns
    -------
    np.array of shape (3, 3)
    """
    xya = homogeneous_coords(a)
    xyb = homogeneous_coords(b)
    # lin. transf. matrix between a and b xy pos = fit
    fit = np.linalg.lstsq(xya, xyb)
    #print fit
    #print "translation=%s" % fit[2,:2]
    return fit[0]

def remove_translation(m):
    """Remove translation from homogeneous transformation matrix."""
    newm = np.zeros(m.shape)
    newm[:2, :2] = m[:2, :2]
    newm[2, 2] = 1
    return newm

def decomp(m):
    """Decompose matrix m into translation, scaling, rotation and shear.

    Used:
    * http://math.stackexchange.com/questions/13150/extracting-rotation-scale-values-from-2d-transformation-matrix?lq=1

    Parameters
    ----------
    m : np.array of shape (3, 3)

    Returns
    -------
    four np.arrays of shape (3, 3)
        Translation, scaling, rotation, and shear matrix. Multiply in that order
        to obtain input matrix: ``np.dot(np.dot(msc, np.dot(mrot, msh)), mtr)``.
    """
    tx = m[2, 0]
    ty = m[2, 1]
    a, b, c, d = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    sx = np.sign(a) * np.sqrt(np.square(a) + np.square(b))
    sy = np.sign(d) * np.sqrt(np.square(c) + np.square(d))
    phi = np.arctan2(-b, a)
    mtr = np.array([[1, 0, 0], [0, 1, 0], [tx, ty, 1]])
    msc = np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])
    mrot = np.array([[np.cos(phi), -np.sin(phi), 0],
                     [np.sin(phi), np.cos(phi), 0],
                     [0, 0, 1]])
    mdecomp = np.dot(np.dot(msc, mrot), mtr)
    msh0 = np.linalg.lstsq(mdecomp[:2, :2], m[:2, :2])
    msh = np.zeros((3, 3))
    msh[:2, :2] += msh0[0]
    msh[2, 2] = 1
    return mtr, msc, mrot, msh

def decomp2(m):
    """Decompose matrix m into translation, scaling, rotation and shear.

    Used:
    * http://math.stackexchange.com/questions/78137/decomposition-of-a-nonsquare-affine-matrix


    Returns
    -------
        mrot.msh.msc.mtr
    """
    c, f = m[2, 0], m[2, 1]
    a, d, b, e = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    p = np.sqrt(a**2 + b**2)
    r = (a*e - b*d) / p
    q = (a*d + b*e) / (a*e - b*d)
    phi = np.arctan2(-b, a)

    mtr = np.array([[1, 0, 0], [0, 1, 0], [c, f, 1]])
    msc = np.array([[p, 0, 0], [0, r, 0], [0, 0, 1]])
    msh = np.array([[1, q, 0], [1, 0, 0], [0, 0, 1]])
    mrot = np.array([[np.cos(phi), -np.sin(phi), 0],
                     [np.sin(phi), np.cos(phi), 0],
                     [0, 0, 1]])

    raise NotImplementedError("Not implemented correctly!")
    return mtr, msc, msh, mrot

def decomp_scalar(m):
    """
    Returns
    -------
    tuple: tx, ty, sx, sy, phi
        phi in rad
    """
    tx = m[2, 0]
    ty = m[2, 1]
    a, b, c, d = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    sx = np.sign(a) * np.sqrt(np.square(a) + np.square(b))
    sy = np.sign(d) * np.sqrt(np.square(c) + np.square(d))
    phi = np.arctan2(-b, a)
    return tx, ty, sx, sy, phi

In [ ]:
# design positions
tx, ty = np.meshgrid(
    np.linspace(-2, 2, 2), 
    np.linspace(-2, 2, 2)
)
txy = np.array([tx.flatten(), ty.flatten()]).T
xy = txy
xy += np.array([1, 1])
xy

In [ ]:
#np.cos(np.pi/4), np.sin(np.pi/4)
theta = np.pi/8
tx, ty = 0.23, 0.98
sx, sy = 1., 1.
m = np.array(
    [[sx*np.cos(theta), -sx*np.sin(theta), 0], 
     [sy*np.sin(theta), sy*np.cos(theta), 0], 
     [tx, ty, 1]]
)
shxy, shyx = 0., 0.
m = np.array(
    [[sx*np.cos(theta) + sx*np.sin(theta)*shxy, -sx*np.sin(theta), 0], 
     [sy*np.sin(theta), sy*np.cos(theta), 0], 
     [tx, ty, 1]]
)

m = np.array(
    [[0.95, 0.86, 0],
     [0.07, 1.13, 0],
     [1.5, -0.52, 1]]
)

m

In [ ]:
mtr, msc, mrot, msh = decomp(m)
tx, ty, sx, sy, phi = decomp_scalar(m)
phi / pi * 180

In [ ]:
mall = np.dot(np.dot(msc, np.dot(mrot, msh)), mtr)
mall, (np.abs(mall - m) < 1e-15).all()
#msh
#np.dot(msc, np.dot(mrot, mtr))

In [ ]:
# measured positions
#m = np.dot(msc, mrot)
xym = np.dot(homogeneous_coords(xy), m)
xym

In [ ]:
# fit of design positions to measured positions
mfit, _, _, _ = np.linalg.lstsq(homogeneous_coords(xy), xym)
mfit

In [ ]:
# fitted positions
xyf = np.dot(homogeneous_coords(xy), mfit)
xyf

In [ ]:
mfittr, mfitsc, mfitrot, mfitsh = decomp(mfit)
mfitall = np.dot(np.dot(mfitsc, np.dot(mfitrot, mfitsh)), mfittr)
mfitall, (np.abs(mfitall - m) < 1e-15).all()

In [ ]:
xyf1 = np.dot(homogeneous_coords(xy), 
              np.dot(np.dot(mfitsc, np.dot(mfitrot, mfitsh)), mfittr))
#xyf1 = np.dot(homogeneous_coords(xy), mfit)

In [ ]:
# deviations
xyc = xym[:, :2] - xy
xyc

In [ ]:
# plot design positions, measured positions, and deviations
fig2 = plt.figure()
fig2.set_size_inches(5, 5)

ax20 = fig2.add_subplot(1, 1, 1)
ax20.scatter(xy[:, 0], xy[:, 1], color="k")
ax20.scatter(xym[:, 0], xym[:, 1], color="r")
#ax20.scatter(xyf1[:, 0], xyf1[:, 1], color="g")
ax20.quiver(xy[:,0], xy[:,1], xyc[:, 0], xyc[:, 1],
            units="xy", scale=1e0)


mtmp0 = np.dot(np.dot(mfitsc, np.dot(mfitrot, mfitsh)), mfittr)
ax20.scatter(np.dot(homogeneous_coords(xy), mtmp0)[:, 0], 
             np.dot(homogeneous_coords(xy), mtmp0)[:, 1], 
             color="pink")

mtmp1 = np.dot(np.dot(mfitsc, mfitrot), mfittr)
ax20.scatter(np.dot(homogeneous_coords(xy), mtmp1)[:, 0], 
             np.dot(homogeneous_coords(xy), mtmp1)[:, 1], 
             color="orange")

mtmp2 = np.dot(mfitrot, mfittr)
ax20.scatter(np.dot(homogeneous_coords(xy), mtmp2)[:, 0], 
             np.dot(homogeneous_coords(xy), mtmp2)[:, 1], 
             color="y")

mtmp3 = mfittr
ax20.scatter(np.dot(homogeneous_coords(xy), mtmp3)[:, 0], 
             np.dot(homogeneous_coords(xy), mtmp3)[:, 1], 
             color="gray")

ax20.set_xlim(-10, 10)
ax20.set_ylim(-10, 10)
ax20.grid()

# Write to DataFrame and csv-file

In [ ]:
df = pd.DataFrame(dtype=np.float64)
df["x pos design"] = xy[:,0]
df["y pos design"] = xy[:,1]
df["x dev"] = xyc[:,0]
df["y dev"] = xyc[:,1]
df.info()

In [ ]:
print df.to_csv(sep=";", float_format="%12.5f")

In [ ]:
#df["y dev"][5] = np.nan
#df["x dev"][4] = np.nan
#print df.to_csv(sep=";", float_format="%12.5f", na_rep=" "*12)